# Lab04 - Evaluation

In [182]:
# Import
import pandas as pd

In [238]:
# Export
export_path = "./export/"

In [241]:
# Utils
def save_latex_table(table, name):
    file_path = export_path + "table_"+ name +".tex"
    with open(file_path, "w") as text_file:
        text_file.write(table.to_latex(index=False))
        print "-> " + file_path + " saved!"
        print "\n"

## Collection loading

In [183]:
coll = pd.read_table("../app/asset/cacm_v2.txt", delimiter="\t", names=["doc_id", "author", "summary", "etc"])
coll.head()

,doc_id,author,summary,etc
0,1,"Perlis, A. J.;Samelson,K.;",Preliminary Report-International Algebraic Lan...,NaN
1,2,"Sugai, I.;",Extraction of Roots by Repeated Subtractions f...,NaN
2,3,"Friedman, M. D.;",Techniques Department on Matrix Program Schemes,NaN
3,4,NaN,Glossary of Computer Engineering and Programmi...,NaN
4,5,"Wadey, W. G.;",Two Square-Root Approximations,NaN


## Query results loading

In [184]:
# True query results
qrels = pd.read_table("../app/asset/qrels.txt", delimiter=" ", names=["query_id", "query_res"])
qrels["relevant"] = True
qrels.head()

,query_id,query_res,relevant
0,1,1410,True
1,1,1572,True
2,1,1605,True
3,1,2020,True
4,1,2358,True


In [216]:
# Query results with the different analyzers
names = ["query_id", "query_res"]
delimiter = " "

results = {
    "whitespace" : pd.read_table("../app/results/qrels_whitespace.txt", delimiter=delimiter, names=names),
    "standard" : pd.read_table("../app/results/qrels_standard.txt", delimiter=delimiter, names=names),
    "english" : pd.read_table("../app/results/qrels_english.txt", delimiter=delimiter, names=names),
    "english_custom" : pd.read_table("../app/results/qrels_english_custom.txt", delimiter=delimiter, names=names)
}

### 1. Summary statistics

In [ ]:
def findRelevant(df):
    df["relevant"] = None
    for idx, row in df.iterrows():
        isRelevant = ((qrels['query_id'] == row.query_id) & 
                      (qrels['query_res'] == row.query_res)).any()
        df.set_value(idx, 'relevant', isRelevant)
    return df

In [235]:
summary = []

for k, v in results.iteritems():
    row = []
    row.append(k)
    row.append(coll.shape[0]) # a. Total number of documents
    row.append(v.shape[0]) # b. total number retrieved documents for all queries
    row.append(qrels.shape[0]) # c. total number of relevant documents for all queries
    v = findRelevant(v)
    nbRelevant = v.loc[v['relevant'] == True].shape[0]
    row.append(nbRelevant) # d. total number of relevant documents retrieved for all queries
    summary.append(row)
    print row

['english', 3204, 82734, 796, 720]
['english_custom', 3204, 142419, 796, 766]
['whitespace', 3204, 121648, 796, 680]
['standard', 3204, 47211, 796, 620]


In [236]:
summary = pd.DataFrame(summary, columns=['Analyzer', 'a.', 'b.', 'c.', 'd.'])
summary

,Analyzer,a.,b.,c.,d.
0,english,3204,82734,796,720
1,english_custom,3204,142419,796,766
2,whitespace,3204,121648,796,680
3,standard,3204,47211,796,620


In [243]:
summary.to_csv(export_path+"summary_statistics.csv")
save_latex_table(summary, "summary")

-> ./export/table_summary.tex saved!


